<a href="https://colab.research.google.com/github/iued-uni-heidelberg/DAAD-Training-2021/blob/main/sentiment_analysis_lexicons_v01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment analysis lexicons using Twitter feeds

In [1]:
!wget https://heibox.uni-heidelberg.de/f/da61b08e7ec945e9b2a5/?dl=1

--2022-02-21 11:26:04--  https://heibox.uni-heidelberg.de/f/da61b08e7ec945e9b2a5/?dl=1
Resolving heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)... 129.206.7.113
Connecting to heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)|129.206.7.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://heibox.uni-heidelberg.de/seafhttp/files/b64feebb-e3ff-4b58-babf-766867cfcf3f/lexicon-overlap-score-examples-master.tar.gz [following]
--2022-02-21 11:26:05--  https://heibox.uni-heidelberg.de/seafhttp/files/b64feebb-e3ff-4b58-babf-766867cfcf3f/lexicon-overlap-score-examples-master.tar.gz
Reusing existing connection to heibox.uni-heidelberg.de:443.
HTTP request sent, awaiting response... 200 OK
Length: 14278919 (14M) [application/octet-stream]
Saving to: ‘index.html?dl=1’

index.html?dl=1     100%[===================>]  13.62M  8.48MB/s    in 1.6s    

2022-02-21 11:26:07 (8.48 MB/s) - ‘index.html?dl=1’ saved [14278919/14278919]



In [ ]:
!mv index.html?dl=1 lexicon-overlap-score-examples-master.tar.gz
!tar -xvzf lexicon-overlap-score-examples-master.tar.gz

In [ ]:
# -*- coding: utf-8 -*-
# Python script to open each file, read json input and copy to one text file for subsequent processing
import os, re, sys

class clLex2Dict(object):
    '''
    @author Bogdan Babych, IÜD, Heidelberg University, 2021
    @email bogdan [dot] babych [at] iued [dot] uni-heidelberg [dot] de
    '''
    def __init__(self, SDirName, output_file = 'lexicon-all.txt'): # initialising by openning the directories
        self.SOutput_file = output_file
        self.openDir(SDirName)
        return

    def openDir(self, path): # implementation of recursively openning directories from a given rule directory and reading each file recursively into a string
        i = 0
        FOut = open(self.SOutput_file, 'w')

        for root,d_names,f_names in os.walk(path):
            for f in f_names:
                ## remove this if using on another corpus


                fullpath = os.path.join(root, f)
                i+=1
                if i%1==0: 
                    print(str(i) + '. Processing: ' + f)
                    print(fullpath)

                # FIn = open(fullpath,'r')
                # SIn = FIn.read()
                # apply text filter, if not None
                # if self.STextFilter and (re.search(self.RFilter, SIn) == None): continue
                # SText2Write = self.procFile(SIn,f,i)

                SText2Write = self.procFile(fullpath, f, i) # returns converted string + tags
                if SText2Write: FOut.write(SText2Write) # if the string is not empty then write to file
                # FIn.close()

                try:
                    pass
                except:
                    print(f'file {f} cannot be read or processed')
                finally:
                    pass
        
        FOut.flush()
        FOut.close()

        return


    def procFile(self, fullpath, SFNameIn, i): # sending each json string for extraction of text and attaching an correct tags to each output string output string
        STagOpen = '<' + self.STag + ' id="' + self.STag + str(self.ID + i)  + '">\n'
        STagClose = '\n</' + self.STag + '>\n\n'
        SText4Corpus = self.getString(fullpath, SFNameIn)
        if SText4Corpus:
            return STagOpen + SText4Corpus + STagClose
        else:
            print('\tNo data read from: ' + SFNameIn)
            return None


    def getString(self, fullpath, SFNameIn):
        '''
        the function uses system commands to copy, open and extract content of needed files
        '''
        # iconv -c -t UTF-8 < input.txt > output.txt
        # iconv -f UTF-8 -t UTF-8//IGNORE 219624.txt > 219624_V2.txt
        # last works!

        SFNameHTML = SFNameIn + '_1.html'
        SFNameTXT2 = SFNameIn + '_2.txt'
        SFNameTXT3 = SFNameIn + '_3.txt'

        SCommand = 'cp ' + fullpath + ' ./' + SFNameHTML
        os.system(SCommand)

        SCommand2 = 'lynx --dump ./' + SFNameHTML + ' > ' + SFNameTXT2
        os.system(SCommand2)

        # SCommand2 = 'iconv -c -t UTF-8 < ' + SFNameHTML + ' > ' + SFNameHTML2
        SCommand3 = 'iconv -f UTF-8 -t UTF-8//IGNORE ' + SFNameTXT2 + ' > ' + SFNameTXT3
        os.system(SCommand3)

        # stream = os.popen('lynx --dump ./' + SFNameHTML2)
        # SFileContent = stream.read()

        F2Read = open('./' + SFNameTXT3, 'r', encoding="utf8", errors="surrogateescape")
        SFileContent = F2Read.read()

        LFileContent = re.split('\n\n+', SFileContent, flags=re.MULTILINE|re.DOTALL)
        print(len(LFileContent))
        print(LFileContent[0])

        LFileContent0 = []
        for el in LFileContent:
            el = re.sub('\n', ' ', el)
            el = re.sub(' +', ' ', el)
            LFileContent0.append(el)

        SFileContent2 = '\n\n'.join(LFileContent0)
        

        return SFileContent2

# calling the class
OHtmlDir2txt = clHtmlDir2txt('/content/www.aerzteblatt.de')